In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.experimental import enable_halving_search_cv
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, HalvingGridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn import set_config
set_config(enable_metadata_routing=True)

from joblib import parallel_backend
from time import monotonic
from prince import FAMD

from utils.data_processing import load_data, raw_columns, full_dtypes, transform_datetime, df_ua_parser, transform_ipinfo, transform_packetinfo, transform_proxyinfo

In [2]:
data_path = Path("./data")
if data_path.joinpath("first_ml_processing.csv").exists():
    processed_data = pd.read_csv(data_path.joinpath("first_ml_processing.csv"))
else:
    # Must use clean_data function to load data 
    input_data_path = Path("./data/cybersecurity_attacks.csv")
    dtypes = {col: col_type for col, col_type in full_dtypes.items() if col in raw_columns}
    raw_data = load_data(input_data_path, dtype=dtypes)

    datetime_columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "IsWeekend"]
    raw_data[datetime_columns] = transform_datetime(raw_data["Timestamp"])
    device_columns = ["String","Browser Name", "Browser Version", "Browser Minor", "Browser Patch",
                    "Browser Patch Minor", "OS Name", "OS Version", "OS Version Minor",
                    "OS Version Patch", "OS Version Patch Minor", "Device Brand", "Device Model",
                    "Device Type"]
    raw_data[device_columns] = df_ua_parser(raw_data["Device Information"])
    proxy_columns = ["Is Proxy"]
    raw_data[proxy_columns] = transform_proxyinfo(raw_data["Proxy Information"])
    ip_columns = ["Int Source IP", "Int Destination IP", "Global Source IP", "Global Destination IP"]
    raw_data[ip_columns] = transform_ipinfo(raw_data[["Source IP Address", "Destination IP Address"]])
    packet_columns = ["Packet Bin"]
    raw_data[packet_columns] = transform_packetinfo(raw_data["Packet Length"], scale=False)

    processed_data = raw_data.drop(columns=["Payload Data","Timestamp", "String", "Device Information",
                                    "Proxy Information", "Source IP Address", "Destination IP Address"])
    processed_data.to_csv(data_path.joinpath("first_ml_processing.csv"), index=False)

In [3]:
Y_true = processed_data["Attack Type"].copy()
X_dataset = processed_data.copy().drop(columns=["Attack Type", "Browser Patch" , "Browser Patch Minor",
                                                "OS Version", "OS Version Minor", "OS Version Patch", "OS Version Patch Minor",
                                                "Device Type", "User Information", "Geo-location Data"])

# Feature Selection
## PCA Analysis

In [4]:
cat_cols = X_dataset.select_dtypes(include=["category","str"]).columns
num_cols = X_dataset.select_dtypes(include=["int64","float64"]).columns
bool_cols = X_dataset.select_dtypes(include=["bool"]).columns
passthrough_columns = [col for col in X_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OrdinalEncoder())
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "pca__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("pca", PCA(n_components=6, random_state=124)),
            ("classifier", RandomForestClassifier(random_state=124))
        ])

relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the grid search with raw data? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=2 , pre_dispatch="n_jobs")
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [5]:
cat_cols = X_dataset.select_dtypes(include=["category","str"]).columns
num_cols = X_dataset.select_dtypes(include=["int64","float64"]).columns
bool_cols = X_dataset.select_dtypes(include=["bool"]).columns
passthrough_columns = [col for col in X_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "pca__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("pca", PCA(n_components=6, random_state=124)),
            ("classifier", RandomForestClassifier(random_state=124))
        ])

relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the grid search with an ordinal encoder for categorical data? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=2, pre_dispatch="n_jobs")
    with parallel_backend('threading', n_jobs=2):
        gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [6]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor"]
Xcat_dataset[columns] = Xcat_dataset[columns].astype("category")

In [7]:
cat_cols = Xcat_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xcat_dataset.select_dtypes(include=["int64","float64"]).columns
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
passthrough_columns = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [("scaler", StandardScaler())]
    )

categorical_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ])
boolean_transformer = Pipeline([
        ("encoder", TargetEncoder(target_type="binary")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, cat_cols),
            ("num", numeric_transformer, num_cols),
            ("bool", boolean_transformer, bool_cols)
        ]
    )

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "pca__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("pca", PCA(n_components=6, random_state=124)),
            ("classifier", RandomForestClassifier(random_state=124))
        ])
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the PCA analysis with more categorical columns? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xcat_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, error_score="raise")
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

In [8]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor"]
Xcat_dataset[columns] = Xcat_dataset[columns].astype("str")
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
Xcat_dataset[bool_cols] = Xcat_dataset[bool_cols].astype("str")

In [ ]:
cat_cols = Xcat_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xcat_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ]).set_output(transform="pandas")

param_grid = {
    "classifier__n_estimators": [100, 200],
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
    "famd__n_components": [2, 6, 10, 15]
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("famd", FAMD()),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


TypeError: SimpleImputer.__init__() got an unexpected keyword argument 'sparse_output'

In [ ]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the FAMD analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xcat_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=2, pre_dispatch="n_jobs")
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

/usr/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/usr/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


ValueError: 
All the 360 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
360 fits failed with the following error:
Traceback (most recent call last):
  File "/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/pipeline.py", line 613, in fit
    Xt = self._fit(X, y, routed_params, raw_params=params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
xtrain_processed = preprocessor.fit_transform(Xcat_dataset, Y_true)

In [ ]:
Xcat_dataset = X_dataset.copy()
columns = ["Year", "Month", "Day", "Hour", "Minute", "Second", "DayOfWeek", "Browser Version", "Browser Minor"]
Xcat_dataset[columns] = Xcat_dataset[columns].astype("str")
bool_cols = Xcat_dataset.select_dtypes(include=["bool"]).columns
Xcat_dataset[bool_cols] = Xcat_dataset[bool_cols].astype("str")

In [ ]:
cat_cols = Xcat_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xcat_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xcat_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ])

param_grid = {
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


In [ ]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the HalvingGridSearchCV analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xcat_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = HalvingGridSearchCV(pipeline, param_grid, resource = "classifier__n_estimators", min_resources=10 , max_resources=500)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

Time taken to fit the model: 335.10 seconds
Model score: 0.334
Best parameters:  {'classifier__max_depth': None, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 90}


In [10]:
columns = ["Int Source IP", "Int Destination IP", "Source Port", "Destination Port", "Protocol", "Packet Type", "Traffic Type", "Attack Signature"]
Xsim_dataset = X_dataset[columns].copy()
bool_cols = Xsim_dataset.select_dtypes(include=["bool"]).columns
Xsim_dataset[bool_cols] = Xsim_dataset[bool_cols].astype("str")

In [11]:
cat_cols = Xsim_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xsim_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xsim_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ])

param_grid = {
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


In [13]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the simple HalvingGridSearchCV analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xsim_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = HalvingGridSearchCV(pipeline, param_grid, resource = "classifier__n_estimators", min_resources=10 , max_resources=1000, factor=2)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

Time taken to fit the model: 97.31 seconds
Model score: 0.335
Best parameters:  {'classifier__max_depth': 20, 'classifier__min_samples_split': 30, 'classifier__n_estimators': 80}


In [ ]:
columns = ["Source Port", "Destination Port", "Protocol", "Packet Type", "Traffic Type", "Attack Signature", "Network Segment"]
Xsim_dataset = X_dataset[columns].copy()
bool_cols = Xsim_dataset.select_dtypes(include=["bool"]).columns
Xsim_dataset[bool_cols] = Xsim_dataset[bool_cols].astype("str")

In [15]:
cat_cols = Xsim_dataset.select_dtypes(include=["category","str"]).columns
num_cols = Xsim_dataset.select_dtypes(include=["int64","float64"]).columns
passthrough_columns = [col for col in Xsim_dataset.columns if col not in cat_cols and col not in bool_cols and col not in num_cols]
    
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ])

param_grid = {
    "classifier__max_depth": [None, 10, 20],
    "classifier__min_samples_split": [2, 10, 30],
}

pipeline = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("classifier", RandomForestClassifier(random_state=124))
        ])


In [16]:
relaunch =""
while relaunch.lower() not in ["y", "n"]:
    relaunch = input("Do you want to relaunch the simple HalvingGridSearchCV analysis? (y/n) ")
if relaunch.lower() == "y":
    X_train, X_test, y_train, y_test = train_test_split(Xsim_dataset, Y_true, test_size=0.2, stratify=Y_true, random_state=124)
    start_time = monotonic()
    gs = HalvingGridSearchCV(pipeline, param_grid, resource = "classifier__n_estimators", min_resources=10 , max_resources=500)
    gs.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % gs.score(X_test, y_test))
    print("Best parameters: ", gs.best_params_)

Time taken to fit the model: 35.26 seconds
Model score: 0.333
Best parameters:  {'classifier__max_depth': 10, 'classifier__min_samples_split': 30, 'classifier__n_estimators': 90}


## Recursive feature elimination with cross-validation